# Advent of Code 2021 Solutions

In [1]:
import pandas as pd
import numpy as np
from aoc import get_input_data

---
## Day 1

### part 1

In [2]:
data = get_input_data(1, year=2021)
data = list(map(int, data.split('\n')))

prev = np.inf
count = 0

for x in data:
    if x > prev:
        count += 1
    prev = x
count

1121

### part 2

In [3]:
prev = np.inf
count = 0
stride = 3

for i in range(len(data) - stride + 1):
    depths = data[i:i + stride]
    total = sum(depths)
    if total > prev:
        count += 1
    prev = total
count

1065

---
## Day 2

### part 1

In [4]:
data = get_input_data(2, year=2021)
data

data = data.split('\n')

x = 0
y = 0

for step in data:
    direction, val = step.split()
    val = int(val)
    
    if direction == 'forward':
        x += val
    elif direction == 'down':
        y += val
    elif direction == 'up':
        y -= val
print(x * y)

1855814


```
down X increases your aim by X units.
up X decreases your aim by X units.
forward X does two things:
It increases your horizontal position by X units.
It increases your depth by your aim multiplied by X.
```

### part 2

In [5]:
x = 0
y = 0
aim = 0

for step in data:
    direction, val = step.split()
    val = int(val)
    
    if direction == 'forward':
        x += val
        y += val * aim
    elif direction == 'down':
        aim += val
    elif direction == 'up':
        aim -= val
print(x * y)

1845455714


---
## Day 3

In [6]:
from collections import Counter

In [7]:
data = get_input_data(3, year=2021)
data = data.split('\n')


### part 1

In [8]:
gamma = ''
epsilon = ''

for digits in zip(*data):
    counts = Counter(digits)
    (g, _), (e, _) = counts.most_common()
    gamma += g
    epsilon += e

result = int(gamma, 2) * int(epsilon, 2)
result

2498354

In [9]:
def filter_data(data, position, value):
    return [x for x in data if x[position] == value]

### Part 2

In [10]:
result = data[:]

for position in range(len(data[0])):
    digits = [x[position] for x in result]
    counts = Counter(digits)
    (oxygen, oxygen_count), (scrubber, scrubber_count) = counts.most_common()
    if oxygen_count == scrubber_count:
        oxygen = '1'
    result = filter_data(result, position, oxygen)
    if len(result) == 1:
        break

o = int(result[0], 2)
o


3921

In [11]:
result = data[:]

for position in range(len(data[0])):
    digits = [x[position] for x in result]
    counts = Counter(digits)
    (oxygen, oxygen_count), (scrubber, scrubber_count) = counts.most_common()
    if oxygen_count == scrubber_count:
        scrubber = '0'
    result = filter_data(result, position, scrubber)
    if len(result) == 1:
        break

s = int(result[0], 2)
o * s


3277956

---

## Day 4
### Part 1

In [12]:
data = get_input_data(4, year=2021)


In [13]:
# Process data
numbers, *boards = data.split('\n\n')

numbers = list(map(int, numbers.split(',')))
boards = np.array([[list(map(int, y.split()))
                    for y in x.split('\n')]
                   for x in boards])


In [14]:
def mark_results(boards, results, number):
    results[boards == number] = 1
    return results

def check_winner(boards, results):
    full_row = results.all(axis=-1)
    full_column = np.transpose(results, axes=(0, 2, 1)).all(axis=-1)

    if full_row.any():
        mask = full_row.any(axis=-1)
        return boards[mask], results[mask]
    
    if full_column.any():
        mask = full_column.any(axis=-1)
        return boards[mask], results[mask]

    return None

def sum_of_unmarked(board, result):
    return board[result == 0].sum()


In [15]:
results = np.zeros(boards.shape)
score = 0

for n in numbers:
    results = mark_results(boards, results, n)
    winner = check_winner(boards, results)
    if winner:
        board, mask = winner
        score = sum_of_unmarked(board, mask) * n
        break

score


2745

### Part 2

In [16]:
def remove_board(boards, results, objs):
    for obj in objs:
        mask = (boards == obj).all(axis=(1, 2))
        boards = np.delete(boards, mask, axis=0)
        results = np.delete(results, mask, axis=0)
    return boards, results

In [17]:
boards_ = boards.copy()
results = np.zeros(boards_.shape)
score = 0

for n in numbers:
    results = mark_results(boards_, results, n)
    winner = check_winner(boards_, results)
    if winner:
        board, mask = winner
        if len(boards_) > 1:
            boards_, results = remove_board(boards_, results, board)
        else:
            score = sum_of_unmarked(board, mask) * n
            break

score

6594

---

## Day 25

In [18]:
def get_move_mask(arr, direction, axis):
    if direction not in ('>', 'v'):
        raise ValueError('direction must be one of [">", "v"]')
    return (arr == direction) & (np.roll(arr, -1, axis=axis) == '.')


def shift(arr, direction, axis):
    mask = get_move_mask(arr, direction, axis)
    arr = np.where(np.roll(mask, 1, axis=axis), np.roll(arr, 1, axis=axis), arr)
    arr[mask] = '.'
    return arr


def step_shift(arr):
    arr = shift(arr, '>', 1)
    arr = shift(arr, 'v', 0)
    return arr


def arrays_equal(arr1, arr2):
    return np.all(arr1 == arr2)


### Part 1

In [19]:
data = get_input_data(25, year=2021)
A = np.array([list(x) for x in data.split('\n')])
valid_moves = True
step = 1

while valid_moves:
    A_new = step_shift(A)
    valid_moves = not arrays_equal(A, A_new)
    if valid_moves:
        A = A_new
        step += 1

step

520